In [3]:
import pymongo
from tqdm import tqdm
import os
import markdown
import re
from nltk.stem import PorterStemmer
st=PorterStemmer()

/home/li/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [4]:
client = pymongo.MongoClient(host="192.168.0.106", port=29197)
client["github"].authenticate("github", "git332", "github")
db = client["github"]

In [12]:
db1 = db.pypi_require_map
db2 = db.pypi_require_map_detail
db3 = db.pypi_tag_stem_top100

In [35]:
"php".find("php")

0

In [6]:
tag_map = {}
i = 0
for d in tqdm(db1.find()):
    i += 1
    if db2.find_one({"name":d["require"]}):
        d2 = db2.find_one({"name":d["require"]})
        if "keywords" in d2:
            for k in d2["keywords"]:
                if k.find("python") == -1:
                    k = st.stem(k)
                    if k in tag_map:
                        tag_map[k] += d["count"]
                    else:
                        tag_map[k] = d["count"]
# for d in db2.find():
#     i += 1
#     if "keywords" in d:
#         for k in d["keywords"]:
#             if k in tag_map:
#                 tag_map[k] += 1
#             else:
#                 tag_map[k] = 1
                
# for d in db3.find():
#     i += 1
#     if "keywords" in d:
#         for k in d["keywords"]:
#             if k in tag_map:
#                 tag_map[k] += 1
#             else:
#                 tag_map[k] = 1

5371it [00:35, 149.73it/s]


In [10]:
sorted(tag_map.items(), key=lambda d: d[1], reverse=True)[:100]

[('http', 2040),
 ('client', 1472),
 ('request', 1433),
 ('kennethreitz', 1415),
 ('forhuman', 1412),
 ('numpi', 1226),
 ('time', 1093),
 ('test', 962),
 ('jinja', 818),
 ('datetim', 778),
 ('timezon', 767),
 ('django', 751),
 ('wsgi', 712),
 ('data-analysi', 695),
 ('pars', 691),
 ('cli', 682),
 ('imag', 659),
 ('machine-learn', 633),
 ('templat', 609),
 ('librari', 580),
 ('flask', 577),
 ('werkzeug', 551),
 ('data-sci', 538),
 ('c', 492),
 ('template-engin', 476),
 ('network', 458),
 ('panda', 455),
 ('web', 454),
 ('termin', 438),
 ('flexibl', 422),
 ('align', 420),
 ('parallel', 408),
 ('consol', 407),
 ('unittest', 392),
 ('redi', 386),
 ('tzinfo', 369),
 ('olson', 369),
 ('unit-test', 358),
 ('web-framework', 352),
 ('cross-platform', 351),
 ('image-process', 343),
 ('shell', 325),
 ('yaml', 324),
 ('gui', 323),
 ('pil', 322),
 ('pillow', 322),
 ('framework', 320),
 ('asyncio', 320),
 ('progress-bar', 315),
 ('progress', 314),
 ('util', 312),
 ('click', 311),
 ('bar', 311),
 ('e

In [6]:
sorted(tag_map.items(), key=lambda d: d[1], reverse=True)[:100]

[('python', 12465),
 ('http', 2025),
 ('client', 1470),
 ('python-requests', 1454),
 ('requests', 1427),
 ('kennethreitz', 1415),
 ('forhumans', 1412),
 ('numpy', 1226),
 ('time', 1088),
 ('python-3', 836),
 ('jinja', 818),
 ('datetime', 775),
 ('django', 746),
 ('wsgi', 712),
 ('data-analysis', 695),
 ('cli', 679),
 ('parsing', 677),
 ('machine-learning', 633),
 ('templates', 594),
 ('python3', 588),
 ('testing', 579),
 ('library', 577),
 ('flask', 574),
 ('werkzeug', 551),
 ('data-science', 538),
 ('python2', 505),
 ('c', 492),
 ('template-engine', 476),
 ('pandas', 455),
 ('terminal', 438),
 ('flexible', 422),
 ('alignment', 420),
 ('console', 407),
 ('parallel', 402),
 ('timezones', 396),
 ('unittest', 387),
 ('timezone', 371),
 ('tzinfo', 369),
 ('olson', 369),
 ('test', 359),
 ('python-2', 358),
 ('unit-testing', 353),
 ('web-framework', 352),
 ('cross-platform', 351),
 ('image-processing', 343),
 ('image', 330),
 ('yaml', 323),
 ('Imaging', 322),
 ('pil', 322),
 ('pillow', 322),

In [11]:
composer_tag_top100 = sorted(tag_map.items(), key=lambda d: d[1], reverse=True)[:100]

In [13]:
for t in composer_tag_top100:
    db3.insert_one({"name":t[0],"count":t[1]})